# EDA MIMIC-III
<img src="files/mimic-db.png" width=40% />

DB reference: https://mit-lcp.github.io/mimic-schema-spy/tables/procedures_icd.html

E&M basics: https://www.youtube.com/watch?v=yuUEKgMMXxo

## 1. Load files

In [1]:
import re
import os
import ast
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import nltk
from nltk.corpus import stopwords
import string
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gauravdesai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 1.1 Start Spark Session

In [15]:
!echo $JAVA_HOME

/Library/Java/JavaVirtualMachines/jdk1.8.0_11.jdk/Contents/Home/


In [2]:
from pyspark.sql import SparkSession
app_name = "ClickThrough"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

### 1.2 Read all files and convert into dataframe

In [11]:
file_path="../data/physionet.org.nosync/files/mimiciii/1.4"
filename_LIST = os.listdir(file_path)
for filename_LIST_ITEM in filename_LIST:
    if filename_LIST_ITEM.endswith("csv"):
        filename,fileformat = filename_LIST_ITEM.split('.')
        exec(filename+'_DF = sqlContext.read.format("'+fileformat+'").option("header", "true").option("multiline",True).'+
             'option("escape",'+"'"+'"'+"')"+'.load("'+file_path+"/"+filename+'.csv")')
        exec(filename+'_DF.createOrReplaceTempView("'+filename+'")')

Following files were loaded

In [4]:
filename_LIST

['ADMISSIONS.csv',
 'CALLOUT.csv',
 'CAREGIVERS.csv',
 'CHARTEVENTS.csv',
 'CPTEVENTS.csv',
 'DATETIMEEVENTS.csv',
 'DIAGNOSES_ICD.csv',
 'DRGCODES.csv',
 'D_CPT.csv',
 'D_EM_CODES.csv',
 'D_ICD_DIAGNOSES.csv',
 'D_ICD_PROCEDURES.csv',
 'D_ITEMS.csv',
 'D_LABITEMS.csv',
 'D_SERVICES.csv',
 'ICUSTAYS.csv',
 'INPUTEVENTS_CV.csv',
 'INPUTEVENTS_MV.csv',
 'LABEVENTS.csv',
 'MICROBIOLOGYEVENTS.csv',
 'NOTEEVENTS.csv',
 'OUTPUTEVENTS.csv',
 'PATIENTS.csv',
 'PRESCRIPTIONS.csv',
 'PROCEDUREEVENTS_MV.csv',
 'PROCEDURES_ICD.csv',
 'SERVICES.csv',
 'TRANSFERS.csv']

## 2 Explore dataframes

### 2.1 Patient demographics

In [36]:
PATIENTS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,46520,23260.5,13429.311598142216,1,9999
SUBJECT_ID,46520,34425.77287188306,28330.40034288402,10,99999
GENDER,46520,None,None,F,M
DOB,46520,None,None,1800-07-02 00:00:00,2201-07-24 00:00:00
DOD,15759,None,None,2100-06-19 00:00:00,2211-06-10 00:00:00
DOD_HOSP,9974,None,None,2100-06-19 00:00:00,2209-01-08 00:00:00
DOD_SSN,13378,None,None,2100-06-19 00:00:00,2211-06-10 00:00:00
EXPIRE_FLAG,46520,0.33875752364574374,0.47329238222741504,0,1


In [182]:
spark.sql("""
select GENDER, count(distinct SUBJECT_ID) as COUNT, sum(EXPIRE_FLAG) as EXPIRE
from PATIENT group by GENDER
""").toPandas().style.hide_index()

GENDER,COUNT,EXPIRE
F,20399,7235
M,26121,8524


Female patients: 44% of 46,520 patients admitted between June 2001 - October 2012 were female

In [78]:
ADMISSIONS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,58976,29488.5,17025.049074819242,1,9999
SUBJECT_ID,58976,33755.5832881172,28092.726225170416,10,99999
HADM_ID,58976,149970.8095835594,28883.095213439887,100001,199999
ADMITTIME,58976,None,None,2100-06-07 19:59:00,2210-08-17 17:13:00
DISCHTIME,58976,None,None,2100-06-09 17:09:00,2210-08-24 19:43:00
DEATHTIME,5854,None,None,2100-06-19 08:15:00,2208-02-05 11:45:00
ADMISSION_TYPE,58976,None,None,ELECTIVE,URGENT
ADMISSION_LOCATION,58976,None,None,** INFO NOT AVAILABLE **,TRSF WITHIN THIS FACILITY
DISCHARGE_LOCATION,58976,None,None,DEAD/EXPIRED,SNF-MEDICAID ONLY CERTIF


In [183]:
pd.set_option('display.max_columns', None)
spark.sql("""
select NUM_ADMISSION, count(*) as COUNT, round(count(*)/465.20,1) as PERC_COUNT 
from (select count(*) as NUM_ADMISSION from ADMISSIONS group by SUBJECT_ID) group by NUM_ADMISSION order by NUM_ADMISSION
""").toPandas().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
NUM_ADMISSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,31,34,42
COUNT,38983,5160,1342,508,246,113,51,31,26,14,13,8,5,4,1,2,3,1,1,1,1,1,1,1,2,1
PERC_COUNT,83.8,11.1,2.9,1.1,0.5,0.2,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- There are 58,976 admissions between June 2001 - October 2012 with 46,520 patients
- 84% patients only had 1 admission with an avg 1.26 admissions in the 10 year period at this hospital
- 5854 patient deaths occured during stay (12.6%)

In [114]:
spark.sql("""
select DURATION_OF_STAY, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT from (
select DATEDIFF(DISCHTIME,ADMITTIME) as DURATION_OF_STAY from ADMISSIONS) GROUP BY DURATION_OF_STAY order by DURATION_OF_STAY
""").toPandas().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161
DURATION_OF_STAY,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,116,117,118,119,120,121,123,124,125,126,127,129,130,131,132,133,134,135,136,137,138,139,140,141,142,144,145,146,148,149,151,154,156,161,163,164,165,167,169,171,172,174,182,192,202,207,295
COUNT,1096,2306,5514,5083,6277,5163,4422,3868,3342,2686,2245,1867,1560,1358,1218,1062,896,811,696,610,558,498,461,399,389,330,287,285,252,250,209,179,150,137,155,152,135,108,104,109,73,97,107,68,66,60,57,56,44,57,37,46,47,45,33,30,42,37,30,27,22,26,24,23,27,19,22,20,15,16,21,15,15,15,16,16,14,15,12,14,6,13,9,7,14,10,10,12,11,6,9,7,8,6,6,7,3,8,10,5,7,7,7,4,2,3,3,8,4,2,2,4,5,7,3,3,4,3,3,1,3,5,6,2,2,5,2,1,1,3,4,2,1,1,1,4,1,1,1,2,1,2,1,1,1,2,1,1,1,1,1,1,2,2,1,1,1,1,1,1,1,1
PERC_COUNT,1.9,3.9,9.3,8.6,10.6,8.8,7.5,6.6,5.7,4.6,3.8,3.2,2.6,2.3,2.1,1.8,1.5,1.4,1.2,1.0,0.9,0.8,0.8,0.7,0.7,0.6,0.5,0.5,0.4,0.4,0.4,0.3,0.3,0.2,0.3,0.3,0.2,0.2,0.2,0.2,0.1,0.2,0.2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
spark.sql("""select round(sum(DATEDIFF(DISCHTIME,ADMITTIME))/ 58976,1) as AVG_STAY from ADMISSIONS""").show()

+--------+
|AVG_STAY|
+--------+
|    10.1|
+--------+



- Same day discharge only happens in 2% cases
- Most frequent duration of stay is 4 days (10.6%)
- Avg length of stay is 10 days

In [121]:
spark.sql("""
select INSURANCE, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT 
from ADMISSIONS GROUP BY INSURANCE order by COUNT desc
""").toPandas().T

,0,1,2,3,4
INSURANCE,Medicare,Private,Medicaid,Government,Self Pay
COUNT,28215,22582,5785,1783,611
PERC_COUNT,47.8,38.3,9.8,3.0,1.0


- 48% of admissions are covered by medicare while 38% are covered by commercial plans
- Only 13% admissions are covered by Medicaid and other Government plans
- 1% admissions had no coverage

In [124]:
spark.sql("""
select ADMISSION_TYPE, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT 
from ADMISSIONS GROUP BY ADMISSION_TYPE order by COUNT desc
""").toPandas().T

,0,1,2,3
ADMISSION_TYPE,EMERGENCY,NEWBORN,ELECTIVE,URGENT
COUNT,42071,7863,7706,1336
PERC_COUNT,71.3,13.3,13.1,2.3


- 74% of the admissions fall under unplanned medical care (Emergency and Urgent)
- Only 26% represents pre-planned or new born related admissions

In [125]:
spark.sql("""
select ADMISSION_LOCATION, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT 
from ADMISSIONS GROUP BY ADMISSION_LOCATION order by COUNT desc
""").toPandas().T

,0,1,2,3,4,5,6,7,8
ADMISSION_LOCATION,EMERGENCY ROOM ADMIT,PHYS REFERRAL/NORMAL DELI,CLINIC REFERRAL/PREMATURE,TRANSFER FROM HOSP/EXTRAM,TRANSFER FROM SKILLED NUR,** INFO NOT AVAILABLE **,HMO REFERRAL/SICK,TRANSFER FROM OTHER HEALT,TRSF WITHIN THIS FACILITY
COUNT,22754,15079,12032,8456,273,204,102,71,5
PERC_COUNT,38.6,25.6,20.4,14.3,0.5,0.3,0.2,0.1,0.0


In [129]:
CAREGIVERS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,7567,3784.0,2184.54907627791,1,999
CGID,7567,17785.33804678208,2185.76546225744,14001,21570
LABEL,7521,4953.333333333333,4208.126582380019,1390,tap
DESCRIPTION,5155,None,None,Administrator,UCO


There are 7,567 unique health care personnel in the database

#### 2.1.1 Transfer

In [222]:
TRANSFERS_DF.summary().toPandas().T

,0,1,2,3,4,5,6,7
summary,count,mean,stddev,min,25%,50%,75%,max
ROW_ID,261897,130949.0,75603.29606240745,1,65464.0,130959.0,196418.0,99999
SUBJECT_ID,261897,33785.59469944291,28086.73318304506,10,11993.0,24156.0,54003.0,99999
HADM_ID,261897,150024.20647430097,28850.07947690453,100001,125101.0,150026.0,174962.0,199999
ICUSTAY_ID,87721,249933.39702009782,28819.49228717971,200001,225055.0,249954.0,274851.0,299999
DBSOURCE,261656,None,None,both,None,None,None,metavision
EVENTTYPE,261873,None,None,admit,None,None,None,transfer
PREV_CAREUNIT,96009,None,None,CCU,None,None,None,TSICU
CURR_CAREUNIT,96016,None,None,CCU,None,None,None,TSICU
PREV_WARDID,202964,31.7257099781242,18.86483457167442,10,15.0,31.0,52.0,9


In [225]:
spark.sql("""select EVENTTYPE, count(distinct HADM_ID) as COUNT, round(count(distinct HADM_ID)/589.76,1) as PERC_COUNT 
from TRANSFERS group by EVENTTYPE""").show()

+---------+-----+----------+
|EVENTTYPE|COUNT|PERC_COUNT|
+---------+-----+----------+
|     null|   24|       0.0|
|    admit|58909|      99.9|
|discharge|58919|      99.9|
| transfer|53858|      91.3|
+---------+-----+----------+



In [295]:
spark.sql("""
select ICU_DAYS, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT from 
(select case when TOTAL_LOS_DAYS=0 or TOTAL_LOS_DAYS is null then 'No ICU Stay' 
when TOTAL_LOS_DAYS!=0 and TOTAL_LOS_DAYS between 0 and 24 then '0-1 days'
when TOTAL_LOS_DAYS!=24 and TOTAL_LOS_DAYS between 24 and 48 then '1-2 days'
when TOTAL_LOS_DAYS!=48 and TOTAL_LOS_DAYS between 48 and 96 then '2-4 days'
when TOTAL_LOS_DAYS!=96 and TOTAL_LOS_DAYS between 96 and 192 then '4-8 days'
when TOTAL_LOS_DAYS!=192 and TOTAL_LOS_DAYS between 192 and 720 then '8-30 days'
when TOTAL_LOS_DAYS > 720 then '>30 days' end as ICU_DAYS 
from (select sum(case when (CURR_CAREUNIT is not null and CURR_CAREUNIT != 'NWARD') then LOS else 0 end) as TOTAL_LOS_DAYS 
from TRANSFERS group by HADM_ID)) group by ICU_DAYS order by ICU_DAYS""").toPandas().T

,0,1,2,3,4,5,6
ICU_DAYS,0-1 days,1-2 days,2-4 days,4-8 days,8-30 days,>30 days,No ICU Stay
COUNT,11311,15524,13803,8315,7298,1531,1194
PERC_COUNT,19.2,26.3,23.4,14.1,12.4,2.6,2.0


98% of hospital admissions required transfer to a care units

#### 2.1.2 ICU Stay
This is the same table obtained after applying the != 'NWARD' and null filter

In [298]:
ICUSTAYS_DF.summary().toPandas().T

,0,1,2,3,4,5,6,7
summary,count,mean,stddev,min,25%,50%,75%,max
ROW_ID,61532,30766.5,17762.902718868758,1,15378.0,30761.0,46143.0,9999
SUBJECT_ID,61532,33888.6059123708,28127.690913330127,10,12040.0,24276.0,54183.0,99999
HADM_ID,61532,149954.4706494182,28898.895903803314,100001,124940.0,149915.0,174991.0,199999
ICUSTAY_ID,61532,249962.71024832607,28890.5748673448,200001,224933.0,249941.0,274971.0,299999
DBSOURCE,61532,None,None,both,None,None,None,metavision
FIRST_CAREUNIT,61532,None,None,CCU,None,None,None,TSICU
LAST_CAREUNIT,61532,None,None,CCU,None,None,None,TSICU
FIRST_WARDID,61532,33.085662744588184,19.10220425336643,12,14.0,33.0,52.0,7
LAST_WARDID,61532,32.978206461678475,19.028608777462928,12,14.0,33.0,52.0,7


In [310]:
spark.sql("""select round(100*count(distinct HADM_ID)/58976,1) as Perc_Admission_with_ICU_stay from ICUSTAYS""").show()

+----------------------------+
|Perc_Admission_with_ICU_stay|
+----------------------------+
|                        98.0|
+----------------------------+



Above count matches with the TRANSFERS table. Variuos care units in the data are as follows:

In [320]:
spark.sql("""
select case when CURR_CAREUNIT = 'CCU' then 'Coronary care unit'
when CURR_CAREUNIT = 'CSRU' then 'Cardiac surgery recovery unit'
when CURR_CAREUNIT = 'MICU' then 'Medical intensive care unit'
when CURR_CAREUNIT = 'NICU' then 'Neonatal intensive care unit'
when CURR_CAREUNIT = 'SICU' then 'Surgical intensive care unit'
when CURR_CAREUNIT = 'TSICU' then 'Trauma/surgical intensive care unit' end as CAREUNIT, 
round(sum(LOS),1) as TOTAL_LOS_HRS, round(sum(LOS)/72433.64,1) as PERC_LOS from TRANSFERS 
where CURR_CAREUNIT is not null and CURR_CAREUNIT != 'NWARD' group by case when CURR_CAREUNIT = 'CCU' then 'Coronary care unit'
when CURR_CAREUNIT = 'CSRU' then 'Cardiac surgery recovery unit'
when CURR_CAREUNIT = 'MICU' then 'Medical intensive care unit'
when CURR_CAREUNIT = 'NICU' then 'Neonatal intensive care unit'
when CURR_CAREUNIT = 'SICU' then 'Surgical intensive care unit'
when CURR_CAREUNIT = 'TSICU' then 'Trauma/surgical intensive care unit' end order by PERC_LOS desc
""").show(truncate = False)

+-----------------------------------+-------------+--------+
|CAREUNIT                           |TOTAL_LOS_HRS|PERC_LOS|
+-----------------------------------+-------------+--------+
|Medical intensive care unit        |2031504.9    |28.0    |
|Neonatal intensive care unit       |1946449.8    |26.9    |
|Surgical intensive care unit       |1107401.0    |15.3    |
|Cardiac surgery recovery unit      |888335.7     |12.3    |
|Trauma/surgical intensive care unit|637441.7     |8.8     |
|Coronary care unit                 |632230.8     |8.7     |
+-----------------------------------+-------------+--------+



#### 2.1.3 Services

In [358]:
SERVICES_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,73343,36672.0,21172.444733662745,1,9999
SUBJECT_ID,73343,34349.07876688982,28270.167117074725,10,99999
HADM_ID,73343,149963.84620209155,28866.972513077206,100001,199999
TRANSFERTIME,73343,None,None,2100-06-07 20:00:22,2210-08-17 17:13:41
PREV_SERVICE,14668,None,None,CMED,VSURG
CURR_SERVICE,73343,None,None,CMED,VSURG


The <b><u>SERVICES</u></b> table should be used if interested in identifying the type of service a patient is receiving in the hospital. For example, if interested in identifying surgical patients, the recommended method is searching for patients admitted under a surgical service

In [364]:
spark.sql("""
select B.Description, count(distinct HADM_ID) as COUNT, round(count(distinct HADM_ID)/589.76,1) as PERC_COUNT 
from SERVICES A left join D_SERVICES B on A.CURR_SERVICE = B.Service
group by B.Description order by COUNT desc
""").show(truncate = False)

+-----------------------------------------------------------------------------------------+-----+----------+
|Description                                                                              |COUNT|PERC_COUNT|
+-----------------------------------------------------------------------------------------+-----+----------+
|Medical - general service for internal medicine                                          |23815|40.4      |
|Cardiac Medical - for non-surgical cardiac related admissions                            |8915 |15.1      |
|Cardiac Surgery - for surgical cardiac admissions                                        |7822 |13.3      |
|Newborn - infants born at the hospital                                                   |7806 |13.2      |
|Surgical - general surgical service not classified elsewhere                             |5298 |9.0       |
|Neurologic Surgical - surgical, relating to the brain                                    |4032 |6.8       |
|Trauma - injury or

### 2.2 Billing - Diagnosis
Coded data recorded primarily for billing and administrative purposes. Includes Current Procedural Terminology (CPT) codes, Diagnosis-Related Group (DRG) codes, and International Classification of Diseases (ICD) codes

Let's look at the diagnosis ICD table. These ICD codes are generated for billing purposes at the end of the hospital stay

In [133]:
DIAGNOSES_ICD_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,651047,325524.0,187941.22469006167,1,99999
SUBJECT_ID,651047,38971.15975805126,29372.198841354115,10,99999
HADM_ID,651047,150017.74481873045,28878.068648016237,100001,199999
SEQ_NUM,651000,7.9138356374807985,6.072633414653878,1,9
ICD9_CODE,651000,25390.601889657988,28365.292859393663,0030,V9103


In [135]:
DIAGNOSES_ICD_DF.show(5)

+------+----------+-------+-------+---------+
|ROW_ID|SUBJECT_ID|HADM_ID|SEQ_NUM|ICD9_CODE|
+------+----------+-------+-------+---------+
|  1297|       109| 172335|      1|    40301|
|  1298|       109| 172335|      2|      486|
|  1299|       109| 172335|      3|    58281|
|  1300|       109| 172335|      4|     5855|
|  1301|       109| 172335|      5|     4254|
+------+----------+-------+-------+---------+
only showing top 5 rows



Create diagnosis grouping for ICD 9 codes

In [184]:
spark.sql("""
select A.*, B.SHORT_TITLE, B.LONG_TITLE, 
case when substr(A.ICD9_CODE,1,1) in ('E','V') then 'external causes of injury and supplemental classification'
when substr(A.ICD9_CODE,1,3) between 001 and 139 then 'infectious and parasitic diseases'
when substr(A.ICD9_CODE,1,3) between 140 and 239 then 'neoplasms'
when substr(A.ICD9_CODE,1,3) between 240 and 279 then 'endocrine, nutritional and metabolic diseases, and immunity disorders'
when substr(A.ICD9_CODE,1,3) between 280 and 289 then 'diseases of the blood and blood-forming organs'
when substr(A.ICD9_CODE,1,3) between 290 and 319 then 'mental disorders'
when substr(A.ICD9_CODE,1,3) between 320 and 389 then 'diseases of the nervous system and sense organs'
when substr(A.ICD9_CODE,1,3) between 390 and 459 then 'diseases of the circulatory system'
when substr(A.ICD9_CODE,1,3) between 460 and 519 then 'diseases of the respiratory system'
when substr(A.ICD9_CODE,1,3) between 520 and 579 then 'diseases of the digestive system'
when substr(A.ICD9_CODE,1,3) between 580 and 629 then 'diseases of the genitourinary system'
when substr(A.ICD9_CODE,1,3) between 630 and 679 then 'complications of pregnancy, childbirth, and the puerperium'
when substr(A.ICD9_CODE,1,3) between 680 and 709 then 'diseases of the skin and subcutaneous tissue'
when substr(A.ICD9_CODE,1,3) between 710 and 739 then 'diseases of the musculoskeletal system and connective tissue'
when substr(A.ICD9_CODE,1,3) between 740 and 759 then 'congenital anomalies'
when substr(A.ICD9_CODE,1,3) between 760 and 779 then 'certain conditions originating in the perinatal period'
when substr(A.ICD9_CODE,1,3) between 780 and 799 then 'symptoms, signs, and ill-defined conditions'
when substr(A.ICD9_CODE,1,3) between 800 and 999 then 'injury and poisoning' 
end as ICD_GROUP
from DIAGNOSES_ICD A
left join D_ICD_DIAGNOSES B
on A.ICD9_CODE = B.ICD9_CODE
""").createOrReplaceTempView('DIAGNOSES_ICD_WITH_GROUPING')

In [204]:
spark.sql("""
select ICD_GROUP, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT from (
select distinct HADM_ID, ICD_GROUP from DIAGNOSES_ICD_WITH_GROUPING) GROUP BY ICD_GROUP order by COUNT desc
""").show(truncate=False)

+---------------------------------------------------------------------+-----+----------+
|ICD_GROUP                                                            |COUNT|PERC_COUNT|
+---------------------------------------------------------------------+-----+----------+
|diseases of the circulatory system                                   |42650|72.3      |
|external causes of injury and supplemental classification            |41887|71.0      |
|endocrine, nutritional and metabolic diseases, and immunity disorders|35606|60.4      |
|diseases of the respiratory system                                   |25260|42.8      |
|injury and poisoning                                                 |22686|38.5      |
|diseases of the genitourinary system                                 |21779|36.9      |
|diseases of the digestive system                                     |21010|35.6      |
|symptoms, signs, and ill-defined conditions                          |20182|34.2      |
|diseases of the bloo

Grouping of ICD 9 reference on Wiki here: https://en.wikipedia.org/wiki/List_of_ICD-9_codes
- Injury from external causes and Cardio vascular issues are the most prominant reasons for hospitalization in the data which is representative of the entire country
- 34% of admissions has ill-defined conditions that couldn't be coded with a specific diagnosis code, let's see how many don't have any other specific diagnosis done by the time of discharge

In [186]:
unspecified_icd_df = spark.sql("""
select SUBJECT_ID, HADM_ID, SHORT_TITLE, ICD_GROUP from DIAGNOSES_ICD_WITH_GROUPING where HADM_ID in 
(select distinct HADM_ID from DIAGNOSES_ICD_WITH_GROUPING where ICD_GROUP = 'symptoms, signs, and ill-defined conditions') 
and HADM_ID in (select distinct HADM_ID from 
(select HADM_ID, count(*) as COUNT from DIAGNOSES_ICD_WITH_GROUPING group by HADM_ID) where COUNT = 1)
order by HADM_ID
""")
unspecified_icd_df.show(truncate = False)

+----------+-------+------------------------+-------------------------------------------+
|SUBJECT_ID|HADM_ID|SHORT_TITLE             |ICD_GROUP                                  |
+----------+-------+------------------------+-------------------------------------------+
|25200     |104775 |Headache                |symptoms, signs, and ill-defined conditions|
|72040     |105193 |Headache                |symptoms, signs, and ill-defined conditions|
|20693     |126654 |Convulsions NEC         |symptoms, signs, and ill-defined conditions|
|81429     |130919 |Headache                |symptoms, signs, and ill-defined conditions|
|30583     |156666 |Headache                |symptoms, signs, and ill-defined conditions|
|19025     |157967 |Swelling in head & neck |symptoms, signs, and ill-defined conditions|
|76949     |173111 |Dysphagia NOS           |symptoms, signs, and ill-defined conditions|
|51486     |182682 |Abdmnal pain rt upr quad|symptoms, signs, and ill-defined conditions|
|43692    

Above 9 admissions are the only cases where no specific ICD9 were coded at discharge

#### CPT Events

In [650]:
CPTEVENTS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,573146,286573.5,165453.14302998674,1,99999
SUBJECT_ID,573146,41805.72370565266,29765.856294001853,100,99999
HADM_ID,573146,149994.68497206646,28835.580959386672,100001,199999
COSTCENTER,573146,None,None,ICU,Resp
CHARTDATE,101545,None,None,2100-06-10 00:00:00,2210-08-19 00:00:00
CPT_CD,573146,92319.31793943878,18365.025658271486,0050T,S2900
CPT_NUMBER,573128,92315.7820800938,18373.538310553315,10022,99999
CPT_SUFFIX,22,None,None,T,T
TICKET_ID_SEQ,471601,37.79845038496526,62.98327984546627,1,999


In [652]:
spark.sql("""
select count(distinct HADM_ID) as HADM_COUNT, 58976-count(distinct HADM_ID) as MISS_HADM, 
count(distinct SUBJECT_ID) as SUBJECT_COUNT from CPTEVENTS
""").show()

+----------+---------+-------------+
|HADM_COUNT|MISS_HADM|SUBJECT_COUNT|
+----------+---------+-------------+
|     44148|    14828|        34005|
+----------+---------+-------------+



So we have CPT data for 44K admissions and are missing about 14K which we might not be able to label for the respective complexity

In [690]:
HADM_COMPLEX_DF = spark.sql("""
select CPT_CATEGORY, COMPLEXITY, count(distinct HADM_ID) as HADM_COUNT from 
(select HADM_ID, CPT_CATEGORY, max(CPT_COMPLEXITY) as COMPLEXITY 
from (select DISTINCT A.HADM_ID, B.CPT_CATEGORY, cast(B.CPT_COMPLEXITY as Integer) from CPTEVENTS A
inner join D_EM_CODES B
on A.CPT_CD = B.procedure_code and A.SECTIONHEADER = 'Evaluation and management') group by HADM_ID, CPT_CATEGORY) 
group by CPT_CATEGORY, COMPLEXITY order by COMPLEXITY
""")
HADM_COMPLEX_DF = HADM_COMPLEX_DF.groupBy('CPT_CATEGORY').pivot('COMPLEXITY').sum('HADM_COUNT').fillna(0)
HADM_COMPLEX_DF.createOrReplaceTempView('HADM_COMPLEX')

In [691]:
spark.sql("""
select B.TYPE, A.* from HADM_COMPLEX A
left join (select distinct CPT_CATEGORY, TYPE from D_EM_CODES) B on A.CPT_CATEGORY = B.CPT_CATEGORY
""").show(truncate = False)

+-------------------------------------------------------+------------+-----+----+-----+----+----+
|TYPE                                                   |CPT_CATEGORY|1    |2   |3    |4   |5   |
+-------------------------------------------------------+------------+-----+----+-----+----+----+
|INITIAL HOSPITAL CARE, PER DAY                         |IH          |1057 |5455|9467 |0   |0   |
|CRITICAL CARE                                          |CC          |22646|3993|0    |0   |0   |
|HOSPITAL DISCHARGE DAY MANAGEMENT                      |HD          |6448 |8888|0    |0   |0   |
|SUBSEQUENT HOSPITAL CARE, PER DAY                      |SH          |1660 |7552|23169|0   |0   |
|INPATIENT CONSULTATION FOR A NEW OR ESTABLISHED PATIENT|CONS        |300  |1008|3073 |6138|5611|
+-------------------------------------------------------+------------+-----+----+-----+----+----+



### 2.3 Interventions - Procedure
Procedures such as dialysis, imaging studies, and placement of lines

In [188]:
PROCEDURES_ICD_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,240095,120048.0,69309.60077795835,1,99999
SUBJECT_ID,240095,32851.48147191737,27771.642798004057,10,99999
HADM_ID,240095,150082.94859118265,28918.380367052585,100003,199999
SEQ_NUM,240095,4.388433744975947,3.7725485754388277,1,9
ICD9_CODE,240095,5689.421349882338,3307.8328680774034,0010,9999


5 most frequent procedures

In [208]:
spark.sql("""
select SHORT_TITLE, count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT from (
select distinct A.HADM_ID, A.ICD9_CODE, B.SHORT_TITLE, B.LONG_TITLE from PROCEDURES_ICD A
left join D_ICD_PROCEDURES B on A.ICD9_CODE = B.ICD9_CODE) GROUP BY SHORT_TITLE order by COUNT desc
""").show(5, truncate=False)

+------------------------+-----+----------+
|SHORT_TITLE             |COUNT|PERC_COUNT|
+------------------------+-----+----------+
|Venous cath NEC         |13133|22.3      |
|Insert endotracheal tube|9882 |16.8      |
|Entral infus nutrit sub |9241 |15.7      |
|Cont inv mec ven <96 hrs|8904 |15.1      |
|Packed cell transfusion |7199 |12.2      |
+------------------------+-----+----------+
only showing top 5 rows



In [211]:
spark.sql("""select NUM_PROC ,count(*) as COUNT, round(count(*)/589.76,1) as PERC_COUNT from (select count(*) as NUM_PROC 
from (select distinct HADM_ID, ICD9_CODE from PROCEDURES_ICD) GROUP BY HADM_ID) group by NUM_PROC order by COUNT desc
""").toPandas().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
NUM_PROC,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,29,26,27,28,32,33,31,35,30
COUNT,10328,9243,7347,5498,4534,3950,2970,2258,1807,1141,805,677,455,435,228,170,96,88,64,46,30,16,14,12,8,5,4,4,3,2,2,1,1,1
PERC_COUNT,17.5,15.7,12.5,9.3,7.7,6.7,5.0,3.8,3.1,1.9,1.4,1.1,0.8,0.7,0.4,0.3,0.2,0.1,0.1,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [218]:
spark.sql("""select count(distinct ICD9_CODE) as UNIQUE_PROC_COUNT from PROCEDURES_ICD""").show()

+-----------------+
|UNIQUE_PROC_COUNT|
+-----------------+
|             2032|
+-----------------+



In [220]:
PROCEDUREEVENTS_MV_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,258066,129033.5,74497.38162177274,1,99999
SUBJECT_ID,258066,63487.969480675485,23327.718654369663,10075,99999
HADM_ID,258066,150012.655475731,28774.35106415752,100001,199999
ICUSTAY_ID,257978,250158.92565257504,28826.07815531956,200001,299998
STARTTIME,258066,None,None,2100-06-08 00:21:00,2209-08-07 14:45:00
ENDTIME,258066,None,None,2100-06-08 14:59:00,2209-08-07 16:52:00
ITEMID,258066,224956.8429897778,1386.6705159732169,221214,228286
VALUE,258066,1328.274611955327,3124.4394140983754,-1,9998
VALUEUOM,258066,None,None,None,min


<b><u>Note:</u> Procedure events</b> is sourced from the rangesignals table, which is a way of storing data with start/stop times. All the itemid are of category "procedure". They are all observations documented by nursing staff at the bed side. They're not billing code procedures like in procedure table with ICD9 codes

In [339]:
spark.sql("""select count(distinct HADM_ID) as COUNT_ADM, 
round(count(*)/count(distinct HADM_ID),1) as OBS_PER_ADM, count(distinct ITEMID) as DISTINCT_PROC, 
count(distinct ORDERCATEGORYNAME) as DISTINCT_CATEGORY
from PROCEDUREEVENTS_MV""").show()

+---------+-----------+-------------+-----------------+
|COUNT_ADM|OBS_PER_ADM|DISTINCT_PROC|DISTINCT_CATEGORY|
+---------+-----------+-------------+-----------------+
|    21894|       11.8|          116|               12|
+---------+-----------+-------------+-----------------+



Distinct category of procedures that are tracked in this table for the 21,894 admissions

In [341]:
spark.sql("""
select ORDERCATEGORYNAME, count(distinct HADM_ID) as COUNT, round(count(distinct HADM_ID)/218.94, 1) as PERC_COUNT 
from PROCEDUREEVENTS_MV group by ORDERCATEGORYNAME order by COUNT desc""").show(truncate = False)

+---------------------+-----+----------+
|ORDERCATEGORYNAME    |COUNT|PERC_COUNT|
+---------------------+-----+----------+
|Peripheral Lines     |20888|95.4      |
|Procedures           |15811|72.2      |
|Invasive Lines       |13494|61.6      |
|Imaging              |13098|59.8      |
|Ventilation          |9329 |42.6      |
|Intubation/Extubation|7947 |36.3      |
|Significant Events   |5778 |26.4      |
|Communication        |2642 |12.1      |
|Continuous Procedures|958  |4.4       |
|Dialysis             |511  |2.3       |
|CRRT Filter Change   |76   |0.3       |
|Peritoneal Dialysis  |42   |0.2       |
+---------------------+-----+----------+



Example of most frequent procedures/ tasks

In [344]:
spark.sql("""
select ORDERCATEGORYNAME, LABEL, count(*) as COUNT, round(count(*)/218.94, 1) as PERC_COUNT from 
(select distinct A.HADM_ID, A.ORDERCATEGORYNAME, B.LABEL from PROCEDUREEVENTS_MV A left join D_ITEMS B on A.ITEMID = B.ITEMID) 
group by ORDERCATEGORYNAME, LABEL order by COUNT desc""").show(10, truncate = False)

+---------------------+--------------------+-----+----------+
|ORDERCATEGORYNAME    |LABEL               |COUNT|PERC_COUNT|
+---------------------+--------------------+-----+----------+
|Peripheral Lines     |20 Gauge            |14867|67.9      |
|Peripheral Lines     |18 Gauge            |12742|58.2      |
|Imaging              |Chest X-Ray         |10561|48.2      |
|Invasive Lines       |Arterial Line       |9491 |43.3      |
|Ventilation          |Invasive Ventilation|8860 |40.5      |
|Procedures           |EKG                 |8530 |39.0      |
|Intubation/Extubation|Extubation          |7127 |32.6      |
|Invasive Lines       |Multi Lumen         |6083 |27.8      |
|Procedures           |Nasal Swab          |5522 |25.2      |
|Imaging              |CT scan             |5270 |24.1      |
+---------------------+--------------------+-----+----------+
only showing top 10 rows



<b><u>Diagnosis-related group (DRG)</u></b> is a system to classify hospital cases into one of approximately 500 groups, also referred to as DRGs, expected to have similar hospital resource use. They have been used in the United States since 1983. There is more than one DRG system being used in the United States, but only the MS-DRG (CMS-DRG) system is used by Medicare

In [347]:
spark.sql("""
select count(distinct HADM_ID) as HADM_COUNT, count(distinct SUBJECT_ID) as SUBJECT_COUNT from DRGCODES""").show()

+----------+-------------+
|HADM_COUNT|SUBJECT_COUNT|
+----------+-------------+
|     58890|        46511|
+----------+-------------+



- 46,511 out of 46,520 patients have at least 1 DRG code linked in the DRG file corresponding to a treatment or diagnosis
- 58,890 out of 58,976 total admissions have at 1 DRG code linked

In [349]:
spark.sql("""
select DESCRIPTION, count(distinct HADM_ID) as COUNT, round(count(distinct HADM_ID)/588.90, 1) as PERC_COUNT 
from DRGCODES group by DESCRIPTION order by COUNT desc""").show(10, truncate = False)

+------------------------------------------------------------------------+-----+----------+
|DESCRIPTION                                                             |COUNT|PERC_COUNT|
+------------------------------------------------------------------------+-----+----------+
|NORMAL NEWBORN                                                          |2805 |4.8       |
|Septicemia & Disseminated Infections                                    |2240 |3.8       |
|Neonate, Bwt > 2499g, Normal Newborn Or Neonate W Other Problem         |1763 |3.0       |
|Cardiac Valve Procedures w/o Cardiac Catheterization                    |1446 |2.5       |
|Coronary Bypass w/o Cardiac Cath Or Percutaneous Cardiac Procedure      |1386 |2.4       |
|SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W MCC                      |1226 |2.1       |
|PREMATURITY WITHOUT MAJOR PROBLEMS                                      |1165 |2.0       |
|NEONATES, DIED OR TRANSFERRED TO ANOTHER ACUTE CARE FACILITY            |1098 |

### 2.4 Laboratory
Blood chemistry, hematology, urine analysis, and microbiology test results

In [647]:
LABEVENTS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,27854055,1.3958520520606676E7,8057286.642303601,1,9999999
SUBJECT_ID,27854055,31463.910791373106,27141.76856797905,10,99999
HADM_ID,22245034,149967.51472890534,28862.313947672308,100001,199999
ITEMID,27854055,51062.14300829807,209.26244103475216,50800,51555
CHARTTIME,27854055,None,None,2096-02-26 01:00:00,2210-08-24 05:53:00
VALUE,27852810,78.53270662127257,5512.982094040352,,specimen lipemic
VALUENUM,24932835,78.53219533605154,5512.982410020097,-.1,9999
VALUEUOM,24753806,None,None,,units
FLAG,9844166,None,None,abnormal,delta


In [649]:
MICROBIOLOGYEVENTS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,631726,315863.5,182363.73241455786,1,99999
SUBJECT_ID,631726,33583.45047853025,28155.535856289614,10,99999
HADM_ID,631726,149741.47169342404,28688.858046087447,100001,199999
CHARTDATE,631726,None,None,2089-11-03 00:00:00,2210-08-18 00:00:00
CHARTTIME,589954,None,None,2089-11-03 22:30:00,2210-08-18 18:07:00
SPEC_ITEMID,631647,70048.1948968332,29.767528621555094,70002,70093
SPEC_TYPE_DESC,631726,None,None,ABSCESS,XXX
ORG_ITEMID,326881,80052.27770962521,69.05435395243953,80002,80312
ORG_NAME,328016,None,None,2ND ISOLATE,ZYGOMYCETES


### 2.5 Medications
Administration records of intravenous medications and medication orders

In [187]:
PRESCRIPTIONS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,4156450,2078225.5,1199863.9075241706,1,999999
SUBJECT_ID,4156450,38772.74062024083,29252.891747744863,10,99999
HADM_ID,4156450,150100.0382379194,28917.541447040403,100001,199999
ICUSTAY_ID,2708742,250004.60841527174,28775.28560850372,200001,299999
STARTDATE,4153268,None,None,2045-10-27 00:00:00,2210-08-22 00:00:00
ENDDATE,4151029,None,None,2026-11-17 00:00:00,2210-08-25 00:00:00
DRUG_TYPE,4156450,None,None,ADDITIVE,MAIN
DRUG,4156450,1.1428571428571428,0.3779644730092272,,~<IND> Rituximab
DRUG_NAME_POE,2492216,1.1428571428571428,0.3779644730092272,Cabergoline ( Dostinex),zz


In [654]:
spark.sql("""select count(distinct NDC) as COUNT from PRESCRIPTIONS""").show()

+-----+
|COUNT|
+-----+
| 4204|
+-----+



In [352]:
spark.sql("""
select DRUG, count(*) as COUNT, round(count(*)/41564.50, 1) as PERC_COUNT 
from PRESCRIPTIONS group by DRUG order by COUNT desc""").show(10, truncate = False)

+---------------------------+------+----------+
|DRUG                       |COUNT |PERC_COUNT|
+---------------------------+------+----------+
|Potassium Chloride         |192993|4.6       |
|Insulin                    |143465|3.5       |
|D5W                        |142241|3.4       |
|Furosemide                 |133122|3.2       |
|0.9% Sodium Chloride       |130147|3.1       |
|NS                         |129731|3.1       |
|Magnesium Sulfate          |90427 |2.2       |
|Iso-Osmotic Dextrose       |87005 |2.1       |
|Sodium Chloride 0.9%  Flush|83392 |2.0       |
|Acetaminophen              |78768 |1.9       |
+---------------------------+------+----------+
only showing top 10 rows



In [353]:
spark.sql("""
select DRUG_TYPE, count(*) as COUNT, round(count(*)/41564.50, 1) as PERC_COUNT 
from PRESCRIPTIONS group by DRUG_TYPE order by COUNT desc""").show(10, truncate = False)

+---------+-------+----------+
|DRUG_TYPE|COUNT  |PERC_COUNT|
+---------+-------+----------+
|MAIN     |3216882|77.4      |
|BASE     |925089 |22.3      |
|ADDITIVE |14479  |0.3       |
+---------+-------+----------+



In [356]:
spark.sql("""select round(count(*)/count(distinct HADM_ID),1) as AVG_RX_COUNT from PRESCRIPTIONS""").show()

+------------+
|AVG_RX_COUNT|
+------------+
|        82.8|
+------------+



Per admission there are on a avg 83 drug administrations or Rx orders placed

### 2.6 Notes
Free text notes such as provider progress notes and hospital discharge summaries

In [408]:
NOTEEVENTS_DF.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
ROW_ID,2083180,1041590.5,601362.4112227767,1,999999
SUBJECT_ID,2083180,32425.63804712027,27554.251090248978,10,99999
HADM_ID,1851344,149879.61056940255,28780.589588346716,100001,199999
CHARTDATE,2083180,None,None,2097-12-07,2210-10-01
CHARTTIME,1766614,None,None,2097-12-07 12:55:00,2210-10-01 09:58:00
STORETIME,1246404,None,None,2097-12-07 13:02:00,2209-02-11 14:38:11
CATEGORY,2083180,None,None,Case Management,Social Work
DESCRIPTION,2083180,None,None,' FILM ORTHO AP LOWER EXT,transfere note
CGID,1246404,17716.297867304664,2158.702632781267,14010,21570


In [496]:
spark.sql("""
select count(distinct case when ISERROR==1 then null else HADM_ID end) as HADM_COUNT,
58976-count(distinct case when ISERROR==1 then null else HADM_ID end) as MISS_DISH_SUMM, 
count(distinct SUBJECT_ID) as SUBJECT_COUNT, sum(ISERROR) as ERROR_NOTES from NOTEEVENTS
""").show()

+----------+--------------+-------------+-----------+
|HADM_COUNT|MISS_DISH_SUMM|SUBJECT_COUNT|ERROR_NOTES|
+----------+--------------+-------------+-----------+
|     58361|           615|        46146|      886.0|
+----------+--------------+-------------+-----------+



In [499]:
spark.sql("""
select CATEGORY, count(*) as COUNT, cast(count(*)/ count(distinct HADM_ID) as Integer) as NOTES_PER_HADM,
round(count(*)/20831.80,1) as PERC_COUNT, min(TEXT_LENGTH) as MIN_TEXT_LENGTH, 
max(TEXT_LENGTH) as MAX_TEXT_LENGTH, cast(mean(TEXT_LENGTH) as Integer) as AVG_TEXT_LENGTH
from (SELECT *,length(TEXT) as TEXT_LENGTH from NOTEEVENTS) group by CATEGORY order by COUNT desc
""").show()

+-----------------+------+--------------+----------+---------------+---------------+---------------+
|         CATEGORY| COUNT|NOTES_PER_HADM|PERC_COUNT|MIN_TEXT_LENGTH|MAX_TEXT_LENGTH|AVG_TEXT_LENGTH|
+-----------------+------+--------------+----------+---------------+---------------+---------------+
|    Nursing/other|822497|            23|      39.5|              3|           3970|            799|
|        Radiology|522279|            11|      25.1|            308|          20574|           1740|
|          Nursing|223556|            24|      10.7|              3|          24282|           1785|
|              ECG|209051|             4|      10.0|              3|           1236|            211|
|       Physician |141624|            15|       6.8|              6|          26477|           7135|
|Discharge summary| 59652|             1|       2.9|             54|          55728|           9618|
|             Echo| 45794|             1|       2.2|            282|           6150|       

We know that there are 58,976 admissions but there are only 55,728 discharge summaries. Let's see how many admissions are misisng discharge summaries and if any have duplicates.
- Discharge summaries and physician attending notes are the longets with on an avg ~10k and 7k words respectively
- Nursing and radiology notes are the most repeated notes within each admission
- ECG and Nursing notes are the shortest notes with only ~211 and 800 words

In [498]:
spark.sql("""
select DESCRIPTION, count(*) as COUNT, cast(count(*)/ count(distinct HADM_ID) as Integer) as NOTES_PER_HADM,
round(count(*)/20831.80,1) as PERC_COUNT, min(TEXT_LENGTH) as MIN_TEXT_LENGTH, 
max(TEXT_LENGTH) as MAX_TEXT_LENGTH, cast(mean(TEXT_LENGTH) as Integer) as AVG_TEXT_LENGTH
from (SELECT *,length(TEXT) as TEXT_LENGTH from NOTEEVENTS) group by DESCRIPTION order by COUNT desc
""").show()

+--------------------+-------+--------------+----------+---------------+---------------+---------------+
|         DESCRIPTION|  COUNT|NOTES_PER_HADM|PERC_COUNT|MIN_TEXT_LENGTH|MAX_TEXT_LENGTH|AVG_TEXT_LENGTH|
+--------------------+-------+--------------+----------+---------------+---------------+---------------+
|              Report|1132519|            19|      54.4|              3|          55728|           1210|
|Nursing Progress ...| 191836|            21|       9.2|              3|          15962|           1678|
| CHEST (PORTABLE AP)| 169270|             5|       8.1|            404|           5320|           1231|
|Physician Residen...|  62698|            12|       3.0|              7|          25397|           7216|
|    CHEST (PA & LAT)|  43158|             3|       2.1|            416|           5257|           1220|
|CT HEAD W/O CONTRAST|  34485|             2|       1.7|            497|           9225|           1903|
|Respiratory Care ...|  31105|             7|       1.5

In [495]:
spark.sql("""
select COUNT AS NUM_DISH_SUMM, count(*) as NUM_HADM from 
(select HADM_ID, count(*) as COUNT from NOTEEVENTS where CATEGORY= 'Discharge summary' group by HADM_ID) group by COUNT
order by NUM_DISH_SUMM""").toPandas().T

,0,1,2,3,4,5,6
NUM_DISH_SUMM,1,2,3,4,5,6,7
NUM_HADM,47006,4782,732,161,30,13,2


So we see that there are cases with more than one discharge note per admission
- More notes in section 3 shows some examples of these notes
- Patient/ physician names, and some other personal identifiable are anonymised
- On an avg there are 19 report summaries/ admission
- Nursing notes are the most recurring category of notes for any admission

In [504]:
spark.sql("""
select CATEGORY, DESCRIPTION, count(*) as COUNT, round(count(*)/20831.80) as PERC_COUNT 
from NOTEEVENTS group by CATEGORY, DESCRIPTION order by COUNT desc""").show(5)

+-------------+--------------------+------+----------+
|     CATEGORY|         DESCRIPTION| COUNT|PERC_COUNT|
+-------------+--------------------+------+----------+
|Nursing/other|              Report|822497|        39|
|          ECG|              Report|209051|        10|
|      Nursing|Nursing Progress ...|191832|         9|
|    Radiology| CHEST (PORTABLE AP)|169270|         8|
|   Physician |Physician Residen...| 62682|         3|
+-------------+--------------------+------+----------+
only showing top 5 rows



## 3. View Input for NLP

### 3.1 Sample Notes: Discharge summary

In [477]:
SAPLE_NOTEEVENTS_DF = spark.sql("""
SELECT CATEGORY,DESCRIPTION,TEXT,length(TEXT) as TEXT_LENGTH from NOTEEVENTS where rand() <= .01
""")

In [488]:
df = SAPLE_NOTEEVENTS_DF.filter("CATEGORY = 'Discharge summary'").filter("TEXT_LENGTH >500").filter("TEXT_LENGTH <1000") \
                    .limit(3).rdd.collect()
for i in df[0:2]:
    print('EXAMPLE '+str(df.index(i)+1))
    print(str(i[2]).encode('utf-8').decode('unicode_escape').replace('\n\n',''))

EXAMPLE 1
CATEGORY: Discharge summary
DESCRIPTION: Report
Admission Date:  [**2150-7-4**]       Discharge Date:  [**2150-7-10**]Date of Birth:   [**2099-6-28**]       Sex:  MService:ADDENDUM:  The patient on [**7-9**] had an episode of desaturation
on his O2 monitoring.  He desated down to 82.  He had a chest
x-ray that was negative.  He had an arterial blood gas, which
revealed a mild respiratory acidosis.  A CTA was performed
and no PEs were found.  The patient remained asymptomatic
throughout this entire procedure.  He has been stable
respiratory wise without difficulty.
                            [**First Name11 (Name Pattern1) **] [**Last Name (NamePattern4) 520**], M.D.  [**MD Number(1) 521**]Dictated By:[**Last Name (NamePattern1) 42727**]
MEDQUIST36D:  [**2150-7-10**]  09:08
T:  [**2150-7-10**]  09:33
JOB#:  [**Job Number 42728**]

EXAMPLE 2
CATEGORY: Discharge summary
DESCRIPTION: Report
Admission Date:  [**2132-1-9**]     Discharge Date:  [**2132-1-22**]Date of Birth:   [**2

### 3.2 Sample Notes: Physician Attending Note

In [695]:
df = SAPLE_NOTEEVENTS_DF.filter("CATEGORY = 'Physician '").filter("TEXT_LENGTH >500").filter("TEXT_LENGTH <1000") \
                    .limit(3).rdd.collect()
for i in df[0:2]:
    print('EXAMPLE '+str(df.index(i)+1))
    print(str(i[2]).encode('utf-8').decode('unicode_escape').replace('\n\n',''))

EXAMPLE 1
TITLE: Attending Note
   I saw and examined the patient, and was physically present with the ICU
   Resident for key portions of the services provided.  I agree with his /
   her note above, including assessment and plan. 70 year old woman with
   end stage COPD, Hflu pneumonia, recent NSTEMI.
   Events:
   Pt
s niece felt pt would not want to be trached. Ongoing process of
   determining HCP status.
   CMV 0.4/500/18/8.
   Tolerated PS10/PEEP8 yesterday for approx 2 hrs.
   Art line removed.
   Exam sig for anasarca, coarse breath sounds, increased wheezing,
   prolonged exp phase, abd soft, affect very depressed but awake and
   responds to questions with nodding.
   plan
     * Hflu pneumonia, beta-lactamase negative, change to ampicillin after
       confirm with lab
     * COPD, continue steroids, MDI
     * Family meeting today
   Total time spent:  30 minutes
   Patient is critically ill.

EXAMPLE 2
Intensivist
   Seen twice tonight.  Ongoing persistent severe hypoxemi

### 3.3 Sample Notes: Nursing/ other Note

In [505]:
df = SAPLE_NOTEEVENTS_DF.filter("CATEGORY = 'Nursing/other'").filter("TEXT_LENGTH >500").filter("TEXT_LENGTH <1000") \
                    .limit(3).rdd.collect()
for i in df[0:2]:
    print('EXAMPLE '+str(df.index(i)+1))
    print(str(i[2]).encode('utf-8').decode('unicode_escape').replace('\n\n',''))

EXAMPLE 1
SOCIAL WORK
SW met with pts husband and son at bedside.  Pts son, [**Name (NI) **], reported that they had met with doctors today to discuss direction of care.  Family states that they are getting a second opinion and then will decide on trach.  Discussed decision making difficulties and quality of life issues. SW offered ongoing support in decision making process.
    Pts husband and son were engaged and talktive throughout interview.  Both presented with sad affect, and a clear understanding of involved medical issues.
     Consulted with RN.  Continue supportive intervention.  Pls pg SW prn.

EXAMPLE 2
S/P FALL/HEAD TRAUMA-T/SICU NPN 11P-7A
S-INTUBATED
O-NEURO-PT DOES NOT OPEN EYES,PERRLA 3-4MM,FLEXES TO NOXIOUS STIM,STRONG COUGH EFFORT WEAK GAG.DOES NOT FOLLOW COMMANDS.
CV-VSS.
RESP-SEE RESP FLOWSHEET FOR VENT DETAILS,NO CHANGES OVER [** **],LS COARSE DECREASED @ BASES,O2 SATS 98%,RR 20'S.VT'S 300'S.
GI/GU-U/O ADEQ CLEAR YELLOW VIA FOLEY,TF IMPACT W/ FIBER @ GOAL 60CC/HR 

### 3.3 Frequently occuring words in Discharge summary

#### 3.3.1 Read NOTES column, convert to lowercase and Tokenize

In [12]:
NOTES_RDD = NOTEEVENTS_DF.filter("CATEGORY = 'Discharge summary'").select('TEXT').rdd.flatMap(lambda x: x)

In [21]:
NOTEEVENTS_DF\
.filter("CATEGORY = 'Discharge summary'")\
.filter("lower(TEXT) LIKE '%birth%'")\
.filter("length(TEXT) <=1000")\
.sample(False,0.1)\
.select("TEXT")\
.first()

Row(TEXT='Admission Date:  [**2114-6-6**]       Discharge Date:\n\nDate of Birth:   [**2047-5-11**]       Sex:  F\n\nService:  CARDIAC SURGERY\n\nDate of discharge pending; awaiting rehabilitation bed.\n\nCHIEF COMPLAINT:  Mitral valve regurgitation.\n\nHISTORY OF PRESENT ILLNESS:  The patient is a 67 year old\nfemale with a history of paroxysmal atrial fibrillation who\npresented to outside hospital two days prior to admission\nhere with a three day history of severe shortness of breath\non exertion.  This has been present for the past one year and\nassociated with chest tightness but no pain.\n\n\n\n\n                            [**Doctor Last Name 412**] [**Last Name (Prefixes) 413**], M.D.  [**MD Number(1) 414**]\n\nDictated By:[**Last Name (NamePattern1) 2209**]\nMEDQUIST36\n\nD:  [**2114-6-23**]  14:37\nT:  [**2114-6-23**]  15:57\nJOB#:  [**Job Number 25407**]\n')

In [6]:
def TokenizeFunct(x):
    if x != 'TEXT':
        return nltk.word_tokenize(re.sub(r'\[*\*.+\*\*]','',x.lower()))
NOTES_RDD = NOTES_RDD.map(TokenizeFunct)

#### 3.3.2 Remove stopwords, numbers and punctuations

In [7]:
def removeStopWordsFunct(x):
    stop_words=set((stopwords.words('english')))
    filteredSentence = [w for w in x if (w not in stop_words)]
    return filteredSentence
NOTES_RDD = NOTES_RDD.map(removeStopWordsFunct)

In [8]:
def removePunctuationsFunct(x):
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    filtered_num = [s for s in filtered_space if not(s.isnumeric())]
    return filtered_num
NOTES_RDD = NOTES_RDD.map(removePunctuationsFunct)

#### 3.4 Lemmatization and most frequent word

In [9]:
def lemmatizationFunct(x):
    lemmatizer = WordNetLemmatizer()
    finalLem = [lemmatizer.lemmatize(s) for s in x if s not in ['']]
    return finalLem
NOTES_RDD = NOTES_RDD.map(lemmatizationFunct)

In [18]:
freqDistRDD = NOTES_RDD.flatMap(lambda x : nltk.FreqDist(x).most_common()) \
                .map(lambda x: x).reduceByKey(lambda x,y : x+y).sortBy(lambda x: x[1], ascending = False)

In [19]:
freqDistRDD.take(20)

[('tablet', 621488),
 ('patient', 588007),
 ('mg', 577775),
 ('po', 503244),
 ('day', 462292),
 ('blood', 440957),
 ('daily', 371431),
 ('sig', 358812),
 ('discharge', 356758),
 ('one', 343883),
 ('history', 308394),
 ('left', 307255),
 ('right', 250562),
 ('time', 235180),
 ('admission', 227811),
 ('pain', 213900),
 ('normal', 193102),
 ('medication', 191156),
 ('pt', 174805),
 ('chest', 143607)]

It took approximately 25 mins to run frequency distribution but less than a min to tokenize